In [7]:
import pandas as pd
from time import time

In [12]:
reader_iterator = pd.read_csv('./data/CGCS-GraphData.csv',iterator=True, chunksize=1000)

In [13]:
t0 = time()
i = 0
for a_chunk in reader_iterator:
    i = i+1
print(time() - t0)

0.019618988037109375


In [45]:
links = pd.read_csv('./data/CGCS-Template.csv')
nodes = pd.read_csv('./data/CGCS-Template-NodeTypes.csv')
links_type = pd.read_csv('./data/EdgeTypesDescription.csv')
nodes_type = pd.read_csv('./data/NodeTypeDescriptions.csv')
demographic_categories = pd.read_csv('./data/DemographicCategories.csv')

In [129]:
nodes_extended = pd.merge(nodes, nodes_type[['NodeType', 'Description']], left_on='NodeType', right_on='NodeType')
nodes_extended = pd.merge(nodes_extended, demographic_categories, how='left', left_on='NodeID', right_on='NodeID')
nodes_extended.rename(columns={'NodeType': 'node_type_id', 'Description': 'node_type', 'NodeID': 'id', 'Category': 'category'}, inplace=True)

In [130]:
links_extended = pd.merge(links, links_type, left_on='eType', right_on='eType')
links_extended.rename(columns={'eType': 'e_type_id', 'Description': 'e_type', 'Source': 'source', 'Target': 'target', 'Time': 'time',
                               'Weight': 'weight', 'SourceLocation': 'source_location', 'TargetLocation': 'target_location',
                               'SourceLatitude': 'source_latitude', 'SourceLongitude': 'source_longitude',
                               'TargetLatitude': 'target_latirude', 'TargetLongitude': 'target_longitude'},
                      inplace=True)

In [131]:
expenses_by_node = pd.DataFrame(links_extended.loc[links_extended['target'].isin(demographic_categories['NodeID'])].groupby(['source'])['weight'].sum()).reset_index()
expenses_by_node.rename(columns={'source': 'id', 'weight': 'total_expense'}, inplace=True)
incomes_by_node = pd.DataFrame(links_extended.loc[links_extended['source'].isin(demographic_categories['NodeID'])].groupby(['target'])['weight'].sum()).reset_index()
incomes_by_node.rename(columns={'target': 'id', 'weight': 'total_income'}, inplace=True)

In [133]:
nodes_extended = pd.merge(nodes_extended, expenses_by_node, how='left', left_on='id', right_on='id')
nodes_extended = pd.merge(nodes_extended, incomes_by_node, how='left', left_on='id', right_on='id')

In [135]:
nodes_extended.to_json('nodes.json', orient='records', indent=2)
links_extended.to_json('links.json', orient='records', indent=2)
links_extended.loc[links_extended['e_type_id']!=5].to_json('links_no_financial.json', orient='records', indent=2)
nodes_extended.to_csv('nodes.csv', index=False)
links_extended.to_csv('links.csv', index=False)
links_extended.loc[links_extended['e_type_id']!=5].to_csv('links_no_financial.csv')